## Dataset Preparation
- Load and merge Yelp reviews, user, and business data
- Engineer key review-level features: review year, engagement flags, and sentiment labels
- Compute user-level rating stability as a proxy for consistency
- Estimate business maturity from earliest review year and clean city names using fuzzy matching
- Merge all features into a single dataset and export `reviews_working.csv` for downstream modeling

In [ ]:
import pandas as pd
from fuzzywuzzy import fuzz, process

In [ ]:
df_reviews = pd.read_csv('./Cleaned Files/Cleaned_Filtered_Yelp_Reviews.csv')
df_users = pd.read_json('./Yelp JSON/yelp_dataset/yelp_academic_dataset_user.json', lines=True)
df_biz = pd.read_csv('./Cleaned Files/Cleaned-Yelp-Businesses-v4.csv')

In [ ]:
def clean_cities_with_state(city_state_df, threshold=85):
    cleaned = {}
    city_state_tuples = list(city_state_df.itertuples(index=False, name=None))
    for city, state in city_state_tuples:
        existing = [k for k in cleaned.keys() if k[1] == state]
        if existing:
            match, score = process.extractOne(city, [x[0] for x in existing], scorer=fuzz.token_set_ratio)
            if score >= threshold:
                key = next(k for k in existing if k[0] == match)
                cleaned[key].append(city)
                continue
        cleaned[(city, state)] = []
    return cleaned

In [ ]:
df_reviews['year'] = pd.to_datetime(df_reviews['date']).dt.year
df_reviews['pre_2020'] = df_reviews['year'] < 2020
df_reviews['engagement_useful'] = df_reviews['useful'] > 0
df_reviews['engagement_funny'] = df_reviews['funny'] > 0
df_reviews['engagement_cool'] = df_reviews['cool'] > 0
df_reviews['text'] = df_reviews['text'].str.replace(r'[\n\t]', ' ', regex=True).str.split().str.join(' ')

user_rating_stats = df_reviews.groupby('user_id')['stars'].agg(['std', 'count'])
user_rating_stats['std'] = user_rating_stats['std'].fillna(0)
df_users = df_users.merge(user_rating_stats, on='user_id', how='left')

df_biz = df_biz.merge(df_reviews.groupby('business_id')['year'].min().rename('established_year'), on='business_id', how='left')
city_state_counts = df_biz[['city', 'state']].drop_duplicates()
cleaned_cities = clean_cities_with_state(city_state_counts)
city_mapping = {city: main_city[0] for main_city, cities in cleaned_cities.items() for city in [main_city[0]] + cities}
df_biz['clean_city'] = df_biz['city'].map(city_mapping)

merged = df_reviews.merge(df_users[['user_id', 'average_stars', 'std', 'count']].rename(columns={'count': 'user_review_count'}), on='user_id')\
                .merge(df_biz[['business_id', 'stars', 'clean_city', 'state', 'categories', 'established_year']], on='business_id', suffixes=('_review', '_business'))
merged['rating_stability'] = merged['std']
merged['positive_review'] = (merged['stars_review'] >= 4).astype(int)
merged['operation_years'] = merged['year'] - merged['established_year']
merged['established'] = merged['operation_years'] > 2

user_stability = merged[['user_id', 'rating_stability']].drop_duplicates(subset='user_id')
percentiles_user_level = user_stability['rating_stability'].describe(percentiles=[0.5, 0.75, 0.8, 0.9]).apply(lambda x: f"{x:.4f}")
print(f"\nUser-level distribution of rating stability:\n{percentiles_user_level}")

relevant_cols = [
    'review_id', 'user_id', 'business_id', 'stars_review', 'text',
    'pre_2020', 'engagement_useful', 'engagement_funny', 'engagement_cool',
    'clean_city', 'state', 'established', 'rating_stability', 'positive_review'
]
merged = merged[relevant_cols]
display(merged.head(5))
merged.to_csv('reviews_working.csv')


User-level distribution of rating stability:
count    728942.0000
mean          0.3120
std           0.6358
min           0.0000
50%           0.0000
75%           0.0000
80%           0.7071
90%           1.4142
max           2.8284
Name: rating_stability, dtype: object


,review_id,user_id,business_id,stars_review,text,pre_2020,engagement_useful,engagement_funny,engagement_cool,clean_city,state,established,rating_stability,positive_review
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,"If you decide to eat here, just be aware it is...",True,False,False,False,North Wales,PA,False,1.118034,0
1,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,Family diner. Had the buffet. Eclectic assortm...,True,False,False,False,Tucson,AZ,True,0.953463,0
2,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Wow! Yummy, different, delicious. Our favorite...",True,True,False,True,Philadelphia,PA,True,0.000000,1
3,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Cute interior and owner (?) gave us tour of up...,True,True,False,True,New Orleans,LA,False,1.541712,1
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,I am a long term frequent customer of this est...,True,True,True,True,Philadelphia,PA,True,0.000000,0
